<a href="https://www.kaggle.com/code/juzykaggle/catboost-accuracy-0-9558?scriptVersionId=139404906" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv
/kaggle/input/icr-identify-age-related-conditions/greeks.csv
/kaggle/input/icr-identify-age-related-conditions/train.csv
/kaggle/input/icr-identify-age-related-conditions/test.csv


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from catboost import CatBoostClassifier

In [3]:
train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')

In [4]:
train

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,fd3dafe738fd,0.149555,3130.05946,123.763599,9.513984,13.020852,3.499305,0.077343,8.545512,2.804172,...,0.173229,1.26092,0.067730,8.967128,217.148554,8095.932828,24.640462,69.191944,21.978000,0
613,fd895603f071,0.435846,5462.03438,85.200147,46.551007,15.973224,5.979825,0.025882,12.622906,3.777550,...,10.223150,1.24236,0.426699,35.896418,496.994214,3085.308063,29.648928,124.808872,0.145340,0
614,fd8ef6377f76,0.427300,2459.10720,130.138587,55.355778,10.005552,8.070549,0.025578,15.408390,1.229900,...,0.173229,0.49706,0.067730,19.962092,128.896894,6474.652866,26.166072,119.559420,21.978000,0
615,fe1942975e40,0.363205,1263.53524,85.200147,23.685856,8.138688,7.981959,0.025578,7.524588,1.229900,...,9.256996,0.78764,0.670527,24.594488,72.611063,1965.343176,25.116750,37.155112,0.184622,0


In [5]:
for i in train.columns:
    if train[i].isna().sum()>0:
        print(i, train[i].dtype)
        train[i] = train[i].fillna(train[i].median())
        print('Filled')

BQ float64
Filled
CB float64
Filled
CC float64
Filled
DU float64
Filled
EL float64
Filled
FC float64
Filled
FL float64
Filled
FS float64
Filled
GL float64
Filled


In [6]:
X = train.drop(['Id', 'Class'], axis=1)
y = train.Class

In [7]:
cat_feat= ['EJ']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [9]:
clf = CatBoostClassifier(cat_features=cat_feat, iterations=625, l2_leaf_reg=7, depth=5).fit(X_train, y_train, eval_set=(X_test, y_test))

0:	learn: 0.6686604	test: 0.6719960	best: 0.6719960 (0)	total: 59.6ms	remaining: 37.2s
1:	learn: 0.6421692	test: 0.6492973	best: 0.6492973 (1)	total: 63.6ms	remaining: 19.8s
2:	learn: 0.6248647	test: 0.6349530	best: 0.6349530 (2)	total: 67.6ms	remaining: 14s
3:	learn: 0.6022747	test: 0.6162265	best: 0.6162265 (3)	total: 71.6ms	remaining: 11.1s
4:	learn: 0.5816835	test: 0.5987099	best: 0.5987099 (4)	total: 75.6ms	remaining: 9.37s
5:	learn: 0.5633571	test: 0.5850615	best: 0.5850615 (5)	total: 79.6ms	remaining: 8.21s
6:	learn: 0.5442888	test: 0.5685169	best: 0.5685169 (6)	total: 83.9ms	remaining: 7.4s
7:	learn: 0.5285951	test: 0.5568495	best: 0.5568495 (7)	total: 88ms	remaining: 6.79s
8:	learn: 0.5116715	test: 0.5414859	best: 0.5414859 (8)	total: 92.1ms	remaining: 6.3s
9:	learn: 0.4944983	test: 0.5259820	best: 0.5259820 (9)	total: 96.2ms	remaining: 5.92s
10:	learn: 0.4772684	test: 0.5124361	best: 0.5124361 (10)	total: 100ms	remaining: 5.58s
11:	learn: 0.4643757	test: 0.5023714	best: 0.502

In [10]:
clf.score(X_test, y_test)

0.9558823529411765

In [11]:
y_pred = pd.DataFrame(clf.predict_proba(test.drop(['Id'], axis=1)))

In [12]:
submission = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')

In [13]:
submission[['class_0', 'class_1']] = y_pred

In [14]:
submission

,Id,class_0,class_1
0,00eed32682bb,0.754797,0.245203
1,010ebe33f668,0.754797,0.245203
2,02fa521e1838,0.754797,0.245203
3,040e15f562a2,0.754797,0.245203
4,046e85c7cc7f,0.754797,0.245203


In [15]:
submission.to_csv('submission.csv', index=False)